In [316]:
import pandas as pd
import numpy as np


################# helper functions #################

# helper functions
# 返回第d列
def project_data(df, d):
    # Return only the d-th column of INPUT
    return df.iloc[:, d]

# 选择数据
def select_data(df, d, val):
    # SELECT * FROM INPUT WHERE input.d = val
    col_name = df.columns[d]
    return df[df[col_name] == val]

# 去除第一列
def remove_first_dim(df):
    # Remove the first dim of the input
    return df.iloc[:, 1:]

# 取父值所拥有的子值表
def slice_data_dim0(df, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(df, 0, v)
    return remove_first_dim(df_temp)

def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)




In [317]:
def buc_single_tuple(df):
    result = []
    head = df.iloc[0].tolist()
    epoch = 2**(df.shape[1]-1)
    length = df.shape[1]
    for i in range(0, epoch):
        bin = '{0:0b}'.format(i)
        bin = '0'*(length-1-len(bin))+bin
        temp = head[:]
        for j in range(len(bin)):
            if bin[j] == "1":
                temp[j] = 'ALL'
        result.append(temp[:])
    return result

def buc(df):
    if len(df.columns.values) == 1:
        return [[sum(df.iloc[:, -1])]]

    # dims = [dim1, rest_of_dims]
    result = []
    dim1_value = sorted(set([x for x in df.iloc[:, 0]]))
    for v in dim1_value:
        list_v = []
        slice_v = slice_data_dim0(df, v)
        if slice_v.shape[0] == 1:
            v_result = buc_single_tuple(slice_v)
        else:
            v_result = buc(slice_v)
        # 遍历result中每一种可能 加上 dim1的v
        for l in v_result:
            l.insert(0, v)
            list_v.append(l)
        result.extend(list_v)

    # ALL 那一行加入result中
    All_df = remove_first_dim(df)
    if All_df.shape == 1:
        all_result = buc_single_tuple(All_df)
    else:
        all_result = buc(All_df)
    for e in all_result:
        e.insert(0, "ALL")
        result.append(e)
    return result

def buc_rec_optimized(df):
    result_list = buc(df)
    result_pd = pd.DataFrame(result_list, columns = df.columns.values)
    return result_pd

In [318]:
input_data = read_data('./asset/a_.txt')
# print(input_data, type(input_data))
# print(type(input_data.columns.values))
d = {'A':[1,1,2], 'B':[2,3,1],'M':[100,100,20]}
df = pd.DataFrame(data = d)
print(df)
# print(sum(df.iloc[:,-1]))
# print(df.iloc[:, 1:3])
# new_df = pd.DataFrame()
# print(new_df)



   A  B    M
0  1  2  100
1  1  3  100
2  2  1   20


In [319]:
print(df.loc[2,:])

A     2
B     1
M    20
Name: 2, dtype: int64


In [320]:
print(slice_data_dim0(df, 1))

   B    M
0  2  100
1  3  100


In [321]:
buc_rec_optimized(df)

,A,B,M
0,1,2,100
1,1,3,100
2,1,ALL,200
3,2,1,20
4,2,ALL,20
5,ALL,1,20
6,ALL,2,100
7,ALL,3,100
8,ALL,ALL,220


In [322]:
d2 = {'A':[1], 'B':[2],'C':[3], 'M':[100]}
df2 = pd.DataFrame(data = d2)
print(df2.iloc[:, 1])

0    2
Name: B, dtype: int64


In [323]:
buc_single_tuple(df2)

[[1, 2, 3, 100],
 [1, 2, 'ALL', 100],
 [1, 'ALL', 3, 100],
 [1, 'ALL', 'ALL', 100],
 ['ALL', 2, 3, 100],
 ['ALL', 2, 'ALL', 100],
 ['ALL', 'ALL', 3, 100],
 ['ALL', 'ALL', 'ALL', 100]]

In [324]:
print(buc_rec_optimized(df2))

     A    B    C    M
0    1    2    3  100
1    1    2  ALL  100
2    1  ALL    3  100
3    1  ALL  ALL  100
4  ALL    2    3  100
5  ALL    2  ALL  100
6  ALL  ALL    3  100
7  ALL  ALL  ALL  100


In [325]:
a = 8
print('{0:0b}'.format(2))
s = '0'*a
for i in range(8,0,-1):
    print(i)

10
8
7
6
5
4
3
2
1


In [326]:
df3 = pd.DataFrame()
print(df3)

Empty DataFrame
Columns: []
Index: []


In [332]:
if df3.empty:
    print("y")

y
